In [545]:
import pandas as pd
import numpy as np

#pd.set_option('display.max_rows', None)
#coaching_time_compare

In [546]:
#fitbit data
wear_time = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitWearTimeViaHR_merged.csv")
dailyactivity = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyActivity_merged.csv")
dailyFitbitActiveZoneMinutes = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\dailyFitbitActiveZoneMinutes_merged.csv")
dailyHRV = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitDailyHRV_merged.csv")
fitbitBreathingRate = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitBreathingRate_merged.csv")
fitbitSkinTemperature = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\fitbitSkinTemperature_merged.csv")
sleepDay = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepDay_merged.csv")
heartrate_15min = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\heartrate_15min_merged.csv")
sleepStageLogInfo = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\sleepStagesDay_merged.csv")

#survey data
survey_data = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\ABriefInterventionTo_DATA_2024-09-22_1540.csv")
coaching_time = pd.read_excel("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\caoching_time.xlsx")

In [547]:
#handle heart zone

hearratezone = pd.read_csv("D:\\EIC-Code\\00-Python\\2024-08-Objective-identification-on-mental-health_CNN_LSTM\\data\\heartRateZones_merged.csv")
hearratezone.drop(columns=['Min','Max','Calories'], inplace=True)
hearratezone_pivoted = hearratezone.pivot_table(index=['Id', 'ActivityDay'], columns='ZoneName', values='Minutes').reset_index()
hearratezone_pivoted.rename(columns = {'ActivityDay':'date'}, inplace = True)

In [548]:
dailyactivity.rename(columns = {'ActivityDate':'date'}, inplace=True)
wear_time.rename(columns = {'Day':'date'}, inplace=True)
dailyFitbitActiveZoneMinutes.rename(columns = {'Date':'date'}, inplace=True)
dailyHRV.rename(columns = {'SleepDay':'date'}, inplace=True)
fitbitBreathingRate.rename(columns = {'SleepDay':'date'}, inplace = True)
fitbitSkinTemperature.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepDay.rename(columns = {'SleepDay':'date'}, inplace=True)
sleepStageLogInfo.rename(columns = {'SleepDay':'date'}, inplace=True)
heartrate_15min.rename(columns = {'Time':'date'}, inplace=True)

#transfer date
total_data = [wear_time, dailyactivity, dailyFitbitActiveZoneMinutes, dailyHRV, fitbitBreathingRate, 
              fitbitSkinTemperature, sleepDay, sleepStageLogInfo, heartrate_15min, hearratezone_pivoted]
for d in total_data:
    d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
    d['date'] = d['date'].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')

#transfer heart_rate_15 min
heartrate_15min = heartrate_15min.groupby(by = ['Id','date']).mean().reset_index()

C:\Users\karri\AppData\Local\Temp\ipykernel_21152\3171284588.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
C:\Users\karri\AppData\Local\Temp\ipykernel_21152\3171284588.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')
C:\Users\karri\AppData\Local\Temp\ipykernel_21152\3171284588.py:15: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  d['date'] = pd.to_datetime(d['date'], errors = 'coerce')


In [549]:
#delete the date if total minutes wear time smaller than 10
wear_time_larger_10_hours = wear_time[wear_time['TotalMinutesWearTime'] >= 10*60] #10hour, what's the meaning of four days a week, is this for calendar week or four consecutive days
len(wear_time_larger_10_hours['Id'].unique()) #99, the number of total participants

99

In [550]:
merged1 = pd.merge(wear_time_larger_10_hours, dailyactivity, on=['Id','date'], how='left')  # inner join only keeps common dates
merged2 = pd.merge(merged1, sleepDay, on=['Id','date'], how='left')
merged3 = pd.merge(merged2, sleepStageLogInfo, on=['Id','date'], how='left')
merged4 = pd.merge(merged3, heartrate_15min, on=['Id','date'], how='left')
merged5 = pd.merge(merged4, dailyFitbitActiveZoneMinutes, on=['Id','date'], how='left')
merged6 = pd.merge(merged5, dailyHRV, on=['Id','date'], how='left')
merged7 = pd.merge(merged6, fitbitBreathingRate, on=['Id','date'], how='left')
merged8 = pd.merge(merged7, fitbitSkinTemperature, on=['Id','date'], how='left')
merged_final = pd.merge(merged8, hearratezone_pivoted, on=['Id','date'], how='left')

In [551]:
#transfer Id to number only
merged_final['Id'] = [items[-3:] for items in merged_final['Id']]
merged_final['Id'] = [int(digit) for digit in merged_final['Id']]

Survey data, find sepecific coaching time

In [552]:
#survey data preprocessing
survey_data['health_coach_survey_complete'].unique()
survey_data_clean = survey_data[survey_data['health_coach_survey_complete'] == 2]#select the whose health coaching session is completed

time_stamp = ['consent_and_screening_survey_timestamp','baseline_wellness_behavior_survey_timestamp','postsession_survey_timestamp',
              'health_coach_survey_timestamp','weeks_followup_survey_timestamp','weeks_followup_survey_96ac_timestamp']

for time in time_stamp:
    survey_data_clean[time] = pd.to_datetime(survey_data_clean[time], errors = 'coerce' )
    survey_data_clean[time] = survey_data_clean[time].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')

#transfer record ID
survey_data_clean.rename(columns = {'record_id':'Id'}, inplace=True)
survey_data_clean['Id'] = [int(digit) for digit in survey_data_clean['Id']]

C:\Users\karri\AppData\Local\Temp\ipykernel_21152\431148340.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_data_clean[time] = pd.to_datetime(survey_data_clean[time], errors = 'coerce' )
C:\Users\karri\AppData\Local\Temp\ipykernel_21152\431148340.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  survey_data_clean[time] = survey_data_clean[time].dt.date #.strftime('%m/%d/%Y').str.replace('/0', '/')
C:\Users\karri\AppData\Local\Temp\ipykernel_21152\431148340.py:9: SettingWithCopyWarning: 
A valu

In [553]:
#coaching time data processing
coaching_time.rename(columns = {'Health Coach Session':'health_coaching_time', "ID":'Id'}, inplace=True)
coaching_time['health_coaching_time'] = pd.to_datetime(coaching_time['health_coaching_time'], errors = 'coerce' )
coaching_time['health_coaching_time'] = coaching_time['health_coaching_time'].dt.date

In [554]:
#merge it with survey data
survey_data_coaching = pd.merge(survey_data_clean, coaching_time, on=['Id'], how='left')

#math coaching time
time_stamp = ['Id','health_coaching_time', 'health_coach_survey_timestamp', 'postsession_survey_timestamp','weeks_followup_survey_timestamp', 'weeks_followup_survey_96ac_timestamp']
coaching_time_compare = survey_data_coaching[time_stamp]

Merge Fitbit data with coaching time

In [555]:
#add another column Fitbit data
merged_final.insert(loc = 2, column = 'survey_date', value = np.nan)
fitbit_data = merged_final.copy()

In [556]:
#Handle duplicate: 98 and 107 is the same person, 98 has fitbit data, 107 has survey data, we keep them consistent as 107
fitbit_data.loc[fitbit_data['Id'] == 98, 'Id'] = 1071
fitbit_data.loc[fitbit_data['Id'] == 107, 'Id'] = 1072
fitbit_data.loc[fitbit_data['Id'] == 1071, 'Id'] = 107

#delete the participants who did not compplete health coaching session
survey_id = survey_data_clean ['Id'].unique() #participants who complete health coaching session
Fitbit_id = fitbit_data['Id'].unique() #participants who have fitbit data
no_completed_coaching = [] #[32, 80, 91, 163] #

for id in Fitbit_id:
    if id not in survey_id:
        no_completed_coaching.append(id)

fitbit_data = fitbit_data[~fitbit_data['Id'].isin(no_completed_coaching)] 

In [557]:
def assign_survey_date(row, time_lst):
    if pd.notna(time) and time_lst == row['date']:
        return 'coach session'
    elif pd.notna(time) and time_lst > row['date']:
        return 'baseline'
    elif pd.notna(time) and time_lst < row['date']:
        return 'follow-up'
    else:
        return None  # In case no condition matches

In [558]:
def seven_day_average(data):

    length_baseline = len(data)
    #calculate the week value
    #average of total minutes wearatime
    data_mean = data.select_dtypes(include=[np.number]).mean()

    length_baseline = len(data)

    ndays = pd.Series({'ndays': length_baseline}) # add the number of days within this week
    data_mean = pd.concat([data_mean, ndays]) #the data wiht days

    return data_mean
    

In [559]:
def baseline(baseline_data):

    len_baseline = len(baseline_data)

    #calculate the baseline
    if len_baseline != 0:
        
        if len_baseline >= 7:
            all_index = list(baseline_data.index)
            index_last_seven_days = all_index[-7:]
            baseline_last_seven = baseline_data.loc[index_last_seven_days] #revise select data to basline
        else: 
            baseline_last_seven = baseline_data

        baseline_mean = seven_day_average(baseline_last_seven)
        baseline_mean.index = [str(col) + '_baseline' for col in baseline_mean.index] #revise the name
        
        return baseline_mean
    
    else:
        return None


In [560]:
def follow_Up(follow_up_data, coach_time):
    len_follow_up = len(follow_up_data)

    if len_follow_up!=0:
        last_day_date = follow_up_data[-1:]['date'] #last_day_date

        day_difference = (last_day_date.loc[last_day_date.index[0]] - coach_time.loc[coach_time.index[0]]).days
        
        time_loop = int(day_difference/7)+1

        next_7_days = coach_time #start day

        id_average = pd.Series()

        for i in range(1, time_loop+1):
            start_day = next_7_days
            start_index = start_day.index[0]

            next_7_days = start_day + pd.Timedelta(days=7)
            
            next_7day_data = follow_up_data[(follow_up_data['date'] > start_day[start_index]) & (follow_up_data['date']<= next_7_days[start_index])]
            follow_up_mean = seven_day_average(next_7day_data)
            follow_up_mean.drop(follow_up_mean.index[0], inplace = True)
            
            follow_up_mean.index = [str(col) + '_week' + str(i) for col in follow_up_mean.index] #revise the name
            
            id_average = pd.concat([id_average, follow_up_mean])
        
        return id_average

    else:
        return None


In [561]:
#evaluate if there is basline or not
fitbit_id = fitbit_data['Id'].unique()
survey_id = survey_data_coaching['Id'].unique()

for i in range(len(fitbit_id)):

    individual_id = fitbit_id[i]
    if individual_id in survey_id:
        individual_survey_data = survey_data_coaching[survey_data_coaching['Id']==individual_id]
        ind = individual_survey_data.index[0]
        coach_time = individual_survey_data['health_coaching_time'][ind]

        individual_fitbit_data = fitbit_data[fitbit_data['Id']==individual_id]
        individual_fitbit_data['survey_date'] = individual_fitbit_data.apply(assign_survey_date, axis=1, time_lst = coach_time)
        fitbit_data.loc[fitbit_data['Id'] == individual_id, 'survey_date'] = individual_fitbit_data['survey_date'] #replace with new define

C:\Users\karri\AppData\Local\Temp\ipykernel_21152\2702394580.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  individual_fitbit_data['survey_date'] = individual_fitbit_data.apply(assign_survey_date, axis=1, time_lst = coach_time)


In [589]:
#used variables
used_var = ['Id','date','survey_date','TotalMinutesWearTime', 'TotalSteps', 'VeryActiveMinutes', 'FairlyActiveMinutes',
       'LightlyActiveMinutes', 'SedentaryMinutes', 'TotalMinutesAsleep_x', 'TotalTimeInBed_x', 
        'TotalTimeAwake', 'TotalMinutesLight', 'TotalMinutesDeep','TotalMinutesREM', 'RestingHeartRate','Cardio',
       'Fat Burn', 'Out of Range', 'Peak'] #delete 'Above', 'Below', Custom Zone

used_fitbit_data = fitbit_data[used_var]

used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep',
                         'TotalTimeInBed_x':'TotalMinutesinbed',
                         'TotalTimeAwake': 'TotalMinutesAwake'}, inplace = True)

all_ids = used_fitbit_data['Id'].unique()

fitbit_final_output = pd.DataFrame()

#main loop
for id in all_ids:
    select_data = used_fitbit_data[used_fitbit_data['Id'] == id]
    coach_time = coaching_time_compare[coaching_time_compare['Id'] == id]['health_coaching_time']

    baseline_data = select_data[select_data['survey_date'] == 'baseline']
    follow_up_data = select_data[select_data['survey_date'] == 'follow-up']

    baseline_mean = baseline(baseline_data)
    id_average = follow_Up(follow_up_data, coach_time)

    baseline_follow_up = pd.concat([baseline_mean,id_average])
    trans_baseline_follow_up = pd.DataFrame(baseline_follow_up).T
    
    #trans_baseline_follow_up['id'] = id
    fitbit_final_output = pd.concat([fitbit_final_output, trans_baseline_follow_up])



C:\Users\karri\AppData\Local\Temp\ipykernel_21152\731330305.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  used_fitbit_data.rename(columns = {'TotalMinutesAsleep_x':'TotalMinutesAsleep',


In [590]:
fitbit_final_output.to_csv('fitbit_final_output.csv')

In [565]:
len(fitbit_final_output) #95

95

Merge fitbit data with survey data

In [579]:
#drop non-use column name
survey_new_data = survey_data_coaching.drop(columns = ['consent_and_screening_survey_timestamp', 
                                                     'informed_consent', 
                                                     'redcap_survey_identifier', 
                                                        'health_coach_survey_timestamp', 
                                                        'postsession_survey_timestamp',
                                                        'consent_and_screening_survey_complete',
                                                        'initials',
                                                        'Participants Name',
                                                        'initials_follow_up',
                                                        'email_address',
                                                        'name',
                                                        'health_coach_survey_complete',
                                                        'initials_post',
                                                        'postsession_survey_complete'
                                                        ])

In [593]:
fitbit_final_output.rename(columns = {'Id_baseline':'Id'}, inplace=True)

survey_fitbit = pd.merge(survey_new_data, fitbit_final_output, on = ['Id'], how = 'left')

In [595]:
survey_fitbit.to_csv('final_survey_fitbit.csv')

In [489]:
#check if the there is Fitbit data in coaching time
ids = fitbit_data['Id'].unique()
i = 0
for id in ids:
    person_day = fitbit_data[fitbit_data['Id']==id]
    
    if person_day['survey_date'].str.contains("coach session").any():
        coach_date = person_day[person_day['survey_date'] == 'coach session']['date']
        print(id,coach_date)
        
    else:
        i=i+1
        print(id)


print(len(ids),i) #99, 18 no coach time


ids = fitbit_data['Id'].unique()
a = 0
for i in ids:
    len_id = len(fitbit_data[fitbit_data['Id'] == i])
    if len_id >= 60:
        a=a+1
print(a) #70 particiapnts have more than 50 days data; 57 participants have more than 60 days

,Id_baseline,TotalMinutesWearTime_baseline,TotalSteps_baseline,VeryActiveMinutes_baseline,FairlyActiveMinutes_baseline,LightlyActiveMinutes_baseline,SedentaryMinutes_baseline,TotalMinutesAsleep_x_baseline,TotalTimeInBed_x_baseline,TotalTimeAwake_baseline,...,TotalTimeAwake_week23,TotalMinutesLight_week23,TotalMinutesDeep_week23,TotalMinutesREM_week23,RestingHeartRate_week23,Cardio_week23,Fat Burn_week23,Out of Range_week23,Peak_week23,ndays_week23
0,24.0,1370.428571,7503.714286,47.714286,28.857143,145.0,689.0,475.0,529.428571,57.571429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0


,Id_baseline,TotalMinutesWearTime_baseline,TotalSteps_baseline,VeryActiveMinutes_baseline,FairlyActiveMinutes_baseline,LightlyActiveMinutes_baseline,SedentaryMinutes_baseline,TotalMinutesAsleep_x_baseline,TotalTimeInBed_x_baseline,TotalTimeAwake_baseline,...,RestingHeartRate_week7,Above_week7,Below_week7,Cardio_week7,Custom Zone_week7,Fat Burn_week7,Out of Range_week7,Peak_week7,ndays_week7,id
0,113.0,1362.285714,8113.0,28.0,7.142857,165.857143,799.857143,408.714286,439.142857,60.285714,...,63.000000,NaN,NaN,0.000000,NaN,3.000000,1436.000000,0.000000,1.0,167
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,60.909091,NaN,NaN,4.363636,NaN,7.636364,1416.454545,0.090909,11.0,167
